In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
hisse = "THYAO"

In [15]:
url = "https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse=" + hisse
r = requests.get(url)
s = BeautifulSoup(r.text,"html.parser")
secim = s.find("select", id="ddlMaliTabloFirst")
secim2 = s.find("select", id="ddlMaliTabloGroup")

In [16]:
tarihler = []
tarihler1 = []
yıllar = []
donemler = []
tumveri = []

In [17]:
cocuklar = secim.findChildren("option")
grup = secim2.findChild("option")["value"]

for i in cocuklar:
    tarihler.append(i.string.rsplit("/"))
    tarihler1.append(i.string)

tarihler1 = pd.to_datetime(tarihler1, format="mixed").strftime('%m/%Y')

In [18]:
plus = 0
while len(tarihler)%4!=0:
    plus += 1
    tarihler.append([0,0])

In [ ]:
for i in range(0,len(tarihler),4):
    parametreler = (
                        ("companyCode", hisse),
                        ("exchange", "TRY"),
                        ("financialGroup", grup),
                        ("year1",tarihler[i][0]),
                        ("period1",tarihler[i][1]),
                        ("year2",tarihler[i+1][0]),
                        ("period2",tarihler[i+1][1]),
                        ("year3",tarihler[i+2][0]),
                        ("period3" ,tarihler[i+2][1]),
                        ("year4",tarihler[i+3][0]),
                        ("period4",tarihler[i+3][1])
                    )
    
    url2 = "https://www.isyatirim.com.tr/_layouts/15/IsYatirim.Website/Common/Data.aspx/MaliTablo"
    r2 = requests.get(url2, params=parametreler).json()["value"]
    data = pd.DataFrame.from_dict(r2)
    columns = data['itemCode']
    columns.name = "index"
    data = data.drop(["itemCode","itemDescTr","itemDescEng"], axis=1)
    data = data.set_index(columns)
    
    if len(tarihler)-i==4 and plus!=0:
        data = data.iloc[:,:4-plus]
    tumveri.append(data)

In [20]:
tumveri = pd.concat(tumveri, axis=1)

In [21]:
tumveri = tumveri.fillna(0)
tumveri = tumveri.astype('int')

ValueError: invalid literal for int() with base 10: 'Dönen Varlıklar'

In [10]:
tumveri.columns = tarihler1

In [11]:
tumveri.to_excel("{}.xlsx".format(hisse))